In [2]:
import os
import openai
import sqlite3
import telebot
from fuzzywuzzy import process

# Set your OpenAI API key here
openai.api_key = os.environ.get('CHATGPT_API_KEY')

bot_token = os.environ.get('TELEGRAM_API_KEY')
bot = telebot.TeleBot(bot_token)

# Questions dictionary
questions = {
    "buyformyself": {"question": "Hi! Thank you for contacting Glovida RX pharmacy. We would need to ask more questions before proceeding. Are you buying medicine for yourself?","type": "str","yn": [],"answer": None,},
    "drug_allergies": {"question": "What drug allergies do you have? (e.g. paracetamol, nsaid, none etc)", "type": "str","answer": None,"storegpt": "The following text is provided by user after he was asked the question 'What drug allergies do you have?' If the text means 'they do not have a drug allergy', please respond with the single word 'no'. But if the text implies 'yes they do', please ensure that the user also have specified what allergy they have, then copy just the allergy term given by user and send it back. But if the response is strange or the user does not specify the allergy, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: "},
    "existing_conditions": {"question": "Do you have any existing medical conditions? If yes, what conditions?","type": "str","answer": None,"storegpt": "The following text is provided by user after he was asked the question 'Do you have any existing medical conditions? If yes, what conditions?' If the text means 'they do not have a medical condition', please respond with the single word 'no'. But if the text implies 'yes', please ensure that the user also have specified what condition they have, then copy just the condition term given by user and send it back. Else if the response is strange or the user does not specify the condition, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: "},
    "current_medication": {"question": "Are you currently taking any medication for your existing medical condition? If yes, what medications?", "type": "str","answer": None,"storegpt": "The following text is provided by user after he was asked the question 'Are you currently taking any medication for your existing medical condition? If yes, what medications?' If the text means 'they are not taking any medication', please respond with the single word 'no'. But if the text implies 'yes', please ensure that the user also have specified what medication they are taking, then copy just the medication term given by user and send it back. Else if the response is strange or the user does not specify the condition, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: "},
    "smoking": {"question": "Are you currently smoking?", "type": "str","yn": [],"answer": None},
    "vaping": {"question": "Are you currently vaping?", "type": "str","yn": [],"answer": None},
    "alcohol": {"question": "Are you currently taking alcohol?", "type": "str","yn": [],"answer": None},
    "occupation": {"question": "What is your occupation?", "type": "str","answer": None,"storegpt": "The following text is provided by user after he was asked the question 'What is your occupation?'. If the text provided is a valid occupation, then copy just the occupation name given by user and send it back. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: "},
    "recent_travel": {"question": "Have you travelled anywhere recently? If yes, where have you travelled to?", "type": "str","answer": None,"storegpt": "The following text is provided by user after he was asked the question 'Have you travelled anywhere recently? If yes, where have you travelled to?' If the text means 'they have not travelled', please respond with the single word 'no'. But if the text implies 'yes', please ensure that the user also have specified where they travelled to, then copy just the location given by user and send it back. Else if the response is strange or the user does not specify the location, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: "},
    "unwell_contacts": {"question": "Is there anyone around you who is also feeling unwell?", "type": "str","yn": [],"answer": None,},
    "pregnant": {"question": "Are you pregnant?", "type": "str","yn": [],"answer": None},
    "symptoms": {
        "question": "What are your symptoms? (please only write one symptom)",
        "type": "str",
        "storegpt": "The following text is provided by user "
        "after he was asked the question 'What are your symptoms?' The valid responses to this question are: 'headache', 'cold', 'diarrhea', "
        "'cough', 'sore throat', 'fatigue', 'flu', 'allergic rhinitis', 'runny nose', 'blocked nose', 'nausea', 'vomit', 'diarrhea', 'indigestion', "
        "'constipation', 'stomach cramps', 'body pain', 'fever', 'gerd', 'eye infection', 'dry tired eyes', 'insomnia', 'fungal infection', "
        "'mouth ulcer', 'acne pimples'. From infering the text, does it line up with any of the responses? If yes, simply respond by sending that word "
        "back to the user and nothing else, do not use any quotes in your response. If it does not line up with any of the options, then add an '@' sign "
        "to the start of your response and inform user what are the valid responses. User Provided Text: ",
        "answer": None,},
    "medication": {"question": "What medication are you taking for your symptoms?", "type": "str","storegpt": "The following text is provided by user after he was asked the question 'What medication are you taking for your symptoms?' If the text means 'they are not taking any medication', please respond with the single word 'no'. But if the text implies 'yes they are', please ensure that the user also have specified what medication they are taking, then copy just the medication term given by user and send it back. Else if the response is strange or the user does not specify the medicine, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ","answer": None}
}

# Symptom related questions
symptom_questions = {
    "cold_duration": {"condition": "cold", "question": "How long has your cold persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your cold persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "cold_fever": {"condition": "cold", "question": "Do you have a fever?", "type": "str", "yn":[], "answer": None},
    "nasal_discharge": {"condition": "cold", "question": "Is your nasal discharge clear and watery or thick and cloudy?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Is your nasal discharge clear and watery or thick and cloudy?' the text should either be 'clear and watery', or 'thick and cloudy', if it implies 'clear and watery', simply type 'clear and watery' (nothing else) and send it back to the user, same logic for thick and cloud, but if the text does not imply one of the two, please add a '@' at the start of your response and come up with your own question to help them get a valid answer. User Provided Text: ", "answer": None},
    "facial_pain": {"condition": "cold", "question": "Are you experiencing facial or sinus pain?", "type": "str", "yn":[], "answer": None},
    "flu_duration": {"condition": "flu", "question": "How long has your flu persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your flu persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "flu_fever": {"condition": "flu", "question": "Do you have a fever?", "type": "str", "yn":[], "answer": None},
    "cough_duration": {"condition": "cough", "question": "How long has your cough persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your cough persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "cough_phlegm": {"condition": "cough", "question": "Is the cough phlegmy? If yes, what colour is the phlegm?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Is the cough phlegmy? If yes, what colour is the phlegm?' If the text means 'the cough is not phlegmy', please respond with the single word 'no'. But if the text implies 'yes it is phlemgy', please ensure that the user also have specified the color of the phlegm, then copy just the exact colour term given by user and send it back. Else if the response is strange or the user does not specify the colour, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ", "answer": None},
    "cough_blood": {"condition": "cough", "question": "Is there any blood present?", "type": "str", "yn":[], "answer": None},
    "recent_cough": {"condition": "cough", "question": "Has your cough begun recently? (less than 1 day)", "type": "str", "yn":[], "answer": None},
    "severe_cough": {"condition": "cough", "question": "Is the cough severe?", "type": "str", "yn":[], "answer": None},
    "cough_relief": {"condition": "cough", "question": "Were you relieved after coughing?", "type": "str", "yn":[], "answer": None},
    "cough_wheezing": {"condition": "cough", "question": "Does your cough come with wheezing?", "type": "str", "yn":[], "answer": None},
    "dust_exposure": {"condition": "cough", "question": "Have you been exposed to dust particles, irritant fumes or cigarette smoke?", "type": "str", "yn":[], "answer": None},
    "rhinitis_duration": {"condition": "allergic rhinitis", "question": "How long have the allergy symptoms persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your allergy symptoms persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "rhinitis_occurrence": {"condition": "allergic rhinitis", "question": "Does your allergy occur often? (yes if it happens more than once a week)", "type": "str", "yn":[], "answer": None},
    "rhinitis_exposure": {"condition": "allergic rhinitis", "question": "Have you been exposed to pollens, feathers, smoke, molds, dust mites or animal dander?", "type": "str", "yn":[], "answer": None},
    "sore_throat_duration": {"condition": "sore throat", "question": "How long have you had a sore throat?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you had a sore throat?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "pollutant_exposure": {"condition": "sore throat", "question": "Have you been exposed to environmental pollutants (e.g. dust, pollen, etc.)?", "type": "str", "yn":[], "answer": None},
    "runny_nose_duration": {"condition": "runny nose", "question": "How long has the runny nose persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has the runny nose persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "blocked_nose_duration": {"condition": "blocked nose", "question": "How long has the blocked nose persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your blocked nose persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "nausea_duration": {"condition": "nausea", "question": "How long have you been nauseous for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your nausea persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "nausea_timing": {"condition": "nausea", "question": "Which part of the day do you usually feel nauseous during?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'Which part of the day do you usually feel nauseous during?' As long as the text is a description of the time of or part of the day, it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "nausea_trigger": {"condition": "nausea", "question": "What triggers your nausea?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'What triggers your nausea?' If the text means 'they do not know', please respond with the single word 'unsure'. If not, please ensure that the user also have specified what the trigger is, then copy just the trigger term given by user and send it back. But if the response is 'no' or the user simply did not specify the trigger, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ", "answer": None},
    "vomit_color": {"condition": "vomit", "question": "What is the colour of your vomit?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'What is the colour of your vomit?' As long as the text is a valid colour, it is valid. Else if the response is strange or the user does not specify the colour, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ", "answer": None},
    "vomit_duration": {"condition": "vomit", "question": "How long have you been vomiting for?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'How long have you been vomiting for?' As long as the text gives a general idea of the frequency, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "diarrhea_occurrence": {"condition": "diarrhea", "question": "How frequent is your diarrhea?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'How frequent is your diarrhea?' As long as the text is a general idea of the frequency, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "diarrhea_duration": {"condition": "diarrhea", "question": "How long have you been having diarrhea for?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'How long have you been having diarrhea for?' As long as the text is a general idea of the frequency, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "diarrhea_color": {"condition": "diarrhea", "question": "What color is your diarrhea?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'What color is your diarrhea?'. 'Please ensure that the text is indeed a valid colour, then copy just the exact colour term (just the colour term and nothing else) given by user and send it back. Else if the response is strange or the user does not specify the colour, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ", "answer": None},
    "eating_habit_diarrhea": {"condition": "diarrhea", "question": "What are your eating habits like?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'What are your eating habits like?' As long as the text is a general idea of their eating habit, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "abdominal_pain_diarrhea": {"condition": "diarrhea", "question": "Do you experience episodes of lower abdominal pain and swelling?", "type": "str", "yn":[], "answer": None},
    "indigestion_duration": {"condition": "indigestion", "question": "How long has the indigestion persisted?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has the indigestion persisted?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "sufficient_rest": {"condition": "indigestion", "question": "Have you been getting sufficient rest? (at least 7 hours of sleep per day)", "type": "str", "yn":[], "answer": None},
    "active": {"condition": "indigestion", "question": "Have you been staying active?", "type": "str", "yn":[], "answer": None},
    "eating_habit_indigestion": {"condition": "indigestion", "question": "What are your eating habits like?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'What are your eating habits like?' As long as the text is a general idea of their eating habit, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "constipation_duration": {"condition": "constipation", "question": "How long have you been constipated for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you been constipated for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "eating_habit_constipation": {"condition": "constipation", "question": "What are your eating habits like?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'What are your eating habits like?' As long as the text is a general idea of their eating habit, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "abdominal_pain": {"condition": "constipation", "question": "Do you experience episodes of lower abdominal pain and swelling?", "type": "str", "yn":[], "answer": None},
    "cramp_duration": {"condition": "stomach cramps", "question": "How long have you had stomach cramps for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you had stomach cramps for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "inflamed_duration": {"condition": "body pain", "question": "How long have you had pain for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you had pain for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "inflamed_part": {"condition": "body pain", "question": "Which part of your body are you experiencing pain in?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Which part of your body are you experiencing pain in?' As long as the text is a general name of the body part, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "inflamed_spread": {"condition": "body pain", "question": "Does the pain spread throughout joints?", "type": "str", "yn":[], "answer": None},
    "fever": {"condition": "body pain", "question": "Do you have a fever?", "type": "str", "yn":[], "answer": None},
    "heartburn_duration": {"condition": "gerd", "question": "How long have you been experiencing heartburn for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you been experiencing heartburn for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "headache_duration": {"condition": "headache", "question": "How long have you had a headache for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you had a headache for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "headache_part": {"condition": "headache", "question": "Which part of your head is in pain?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Which part of your head is in pain?' As long as the text is a general area of the head, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "eye_infection_duration": {"condition": "eye infection", "question": "How long has your eye been hurting for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your eye been hurting for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "eye_redness_infection": {"condition": "eye infection", "question": "Is your eye red?", "type": "str", "yn":[], "answer": None},
    "eye_discharge": {"condition": "eye infection", "question": "Do you have discharge coming out from your eye? If yes, what is the color of your discharge?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Do you have discharge coming out from your eye? If yes, what is the color of your discharge?' If the text means 'no, there is no discharge', please respond with the single word 'no'. But if the text implies 'yes there is discharge', please ensure that the user also have specified the color of the discharge, then copy just the exact colour term given by user and send it back. Else if the response is strange or the user does not specify the colour, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. User Provided Text: ", "answer": None},
    "blink_pain": {"condition": "eye infection", "question": "Does it hurt when you blink?", "type": "str", "yn":[], "answer": None},
    "tired_eye_duration": {"condition": "dry tired eyes", "question": "How long has your eye been hurting for?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long has your eye been hurting for?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "eye_redness_tired": {"condition": "dry tired eyes", "question": "Is your eye red?", "type": "str", "yn":[], "answer": None},
    "vision_blurry": {"condition": "dry tired eyes", "question": "Is your vision blurry? If yes, how long has it been blurry for?", "type": "str", "yn":[], "answer": None},
    "insomnia_occurrence": {"condition": "insomnia", "question": "How often do you have trouble sleeping?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'How often do you have trouble sleeping?' As long as the text is a general idea of the frequency, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "fungal_duration": {"condition": "fungal infection", "question": "How long have you been having the infection?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you been having the infection?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "fungal_area": {"condition": "fungal infection", "question": "Which part of your body is infected?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Which part of your body is infected?' As long as the text is a general name of the body part, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "ulcer_duration": {"condition": "mouth ulcer", "question": "How long have you been having the ulcer?", "type": "str", "chatgpt": "The following text is provided by user after he was asked the question 'How long have you been having the ulcer?' As long as the text is any sort of duration (minutes, hours, days, weeks etc), it is valid. If not, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "ulcer_description": {"condition": "mouth ulcer", "question": "Could you describe your ulcer?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Can you state the size of your acne bumps?' As long as the text is a general description of size, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "ulcer_appearance": {"condition": "mouth ulcer", "question": "Does it have irregular borders or is uniformly circular?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Does it have irregular borders or is uniformly circular?' the text should either be 'irregular borders', or 'uniformly circular', if it implies 'irregular borders', simply type 'irregular borders' (nothing else) and send it back to the user, same logic for uniformly circular, but if the text does not imply one of the two, please add a '@' at the start of your response and come up with your own question to help them get a valid answer. User Provided Text: ", "answer": None},
    "acne_occurrence": {"condition": "acne pimples", "question": "Do you know roughly when your acne started?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Do you know roughly when your acne started?' As long as the text is a general idea of when it started, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "facial_product": {"condition": "acne pimples", "question": "Have you used any facial products?", "type": "str", "yn":[], "answer": None},
    "acne_size": {"condition": "acne pimples", "question": "Can you state the size of your acne bumps?", "type": "str", "storegpt": "The following text is provided by user after he was asked the question 'Can you state the size of your acne bumps?' As long as the text is a general description of size, it is valid, and you should summarize that response into as little words and possible and send it back to the user (just the summary and nothing else). If it is not valid, please add a '@' at the start of your response and inform user that what they wrote is invalid and come up with a prompt that helps them answer the question. 'Yes' and 'No' are also not valid replies so you should add the '@' if they wrote that. User Provided Text: ", "answer": None},
    "acne_inflamed": {"condition": "acne pimples", "question": "Are they red and inflamed?", "type": "str", "yn":[], "answer": None},
    "acne_pus": {"condition": "acne pimples", "question": "Is there pus on the bumps?", "type": "str", "yn":[], "answer": None}
}


user_responses = {}
current_question_index = {}
current_symptom_question_index = {}

def chatgpt(prompt, context=[]):
    try:
        messages = [{"role": "system", "content": "You are AI chatbot for a pharmacy company tasked to interpret user response to medical related questions."}]
        for item in context:
            messages.append({"role": "user", "content": item['user']})
            messages.append({"role": "assistant", "content": item['assistant']})
        messages.append({"role": "user", "content": prompt})

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None

def validate_with_storegpt(question, response): #suppose to store chatgpt response
    chatgpt_prompt = question.get("storegpt", "")
    if chatgpt_prompt:
        prompt = chatgpt_prompt + response
        corrected_response = chatgpt(prompt).strip().lower()
        if "@" in corrected_response:
            return corrected_response, False
        else:
            return corrected_response, True
    return response, True

def validate_with_chatgpt(question, response): #suppose to store user response
    chatgpt_prompt = question.get("chatgpt", "")
    if chatgpt_prompt:
        prompt = chatgpt_prompt + response
        corrected_response = chatgpt(prompt).strip().lower()
        if "@" in corrected_response:
            return corrected_response, False
        else:
            return response, True
    return response, True

def validate_and_correct_response(question, response, valid_responses):
    prompt = (
        f"User provided the response '{response}' to the question :'{question}'. The valid options are: [{', '.join(valid_responses)}] "
        f"In a medical context, if the meaning of the response is equivalent to one of the valid options, reply with that exact word (all uncapitalized) and nothing else, "
        f"e.g. if user types 'i'm not sure' and one of the options are 'none', simply respond with the word 'none'. "
        f"But if the response does not match any of the options (user gives a strange response), please add the keyword '@' and then ask "
        f"your own questions to prompt user to give you a response that does match an option."
    )
    corrected_response = chatgpt(prompt).strip().lower()
    if "@" in corrected_response:
        return corrected_response, False
    else:
        return corrected_response, True

def validate_yes_or_no(question, response):
    prompt = (
        f"The following text is provided by user after he was asked the question '{question}'. "
        f"The text HAS TO BE exactly either 'yes' or 'no', if it is not 'yes' or 'no' or if the text means they don't know or just means anything other than yes or no, please add '@' to your response and then ask "
        f"your own questions to prompt user to answer 'yes' or 'no'. If text is yes, respond with the single word 'yes', if text is no, respond with the single word 'no'. User Provided Text: '{response}'"
    )
    corrected_response = chatgpt(prompt).strip().lower()
    if "@" in corrected_response:
        return corrected_response, False
    else:
        return corrected_response, True

def validate_and_correct(question, response):
    prompt = (
        # f"The following text is provided by user after he was asked the question '{question}' If the text means "
        # f"'they do not have it or no', please respond with the single word 'no'. But if the text implies 'yes', please ensure that the user also has answered all parts of the question, "
        # f"Else if the response is strange or the user does not specify the allergy, please add a '!' at the start of your response and inform user that what they wrote is invalid and come up with a "
        # f"prompt that helps them answer the question. User Provided Text: '{response}'"
        f"User provided the response '{response}' to the question :'{question}'. Is that a valid response to that question? If yes, simply type the word 'valid'. "
        f"If the question gives the statement: if yes,... it means they are only required to answer the second part if their response was yes. Replying no, is valid"
        f"If user is unsure or gives a strange response, then add a ! sign at the start of the sentence, then craft and ask a short question to prompt user to give you a response that is valid."
        # f"User provided the response '{response}' to the question :'{question}'. "
        # f"if the user response is 'Yes', please ensure that all other parts of the questions are answered, or else it would be considered invalid. The response 'No' itself is also valid."
        # f"else if the user is unsure or gives a strange or invalid response, then add a ! sign at the start of the sentence, then repeat the question."
    )
    corrected_response = chatgpt(prompt).strip().lower()
    if "!" in corrected_response:
        return corrected_response, False
    else:
        return response, True

def validate_and_convert_input(user_input, expected_type):
    if expected_type == "str":
        return str(user_input)
    elif expected_type == "int":
        try:
            return int(user_input)
        except ValueError:
            raise ValueError(f"Invalid input: Expected an integer but got '{user_input}'")
    else:
        raise ValueError(f"Unsupported data type: {expected_type}")

def should_ask_symptom_question(symptom, question_key):
    condition = symptom_questions[question_key]["condition"]
    return symptom == condition

def ask_next_question(chat_id):
    index = current_question_index[chat_id]
    question_keys = list(questions.keys())
    if index < len(question_keys):
        current_question_key = question_keys[index]
        question = questions[current_question_key]["question"]
        bot.send_message(chat_id, question)
    else:
        ask_symptom_related_questions(chat_id)

def ask_symptom_related_questions(chat_id):
    symptom = user_responses[chat_id].get("symptoms", "").lower()
    symptom_question_keys = list(symptom_questions.keys())
    symptom_index = current_symptom_question_index[chat_id]
    if symptom_index < len(symptom_question_keys):
        current_symptom_question_key = symptom_question_keys[symptom_index]
        if should_ask_symptom_question(symptom, current_symptom_question_key):
            question = symptom_questions[current_symptom_question_key]["question"]
            bot.send_message(chat_id, question)
        else:
            current_symptom_question_index[chat_id] += 1
            ask_symptom_related_questions(chat_id)
    else:
        end_of_symptom_questions(chat_id)

def end_of_symptom_questions(chat_id):
    current_symptom_question_index[chat_id] = 0
    bot.send_message(chat_id, "Do you have any other symptoms? If yes, please state one more symptom. If no, type 'no'. Valid symptoms: 'headache', 'cold', 'diarrhea', 'cough', 'sore throat', 'fatigue', 'flu', 'allergic rhinitis', 'runny nose', 'blocked nose', 'nausea', "
        "'vomit', 'diarrhea', 'indigestion', 'constipation', 'stomach cramps', 'body pain', 'fever', 'gerd', 'eye infection', 'dry tired eyes', 'insomnia', 'fungal infection', 'mouth ulcer', 'acne pimples'.")
    current_question_index[chat_id] = -1

def end_of_questions(chat_id, username, user_id):
    summary = "End of questions. Thank you for your responses."
    response_summary = "\n".join([f"{key}: {questions[key]['answer']}" for key in questions.keys()])
    response_summary += "\n" + "\n".join([f"{key}: {symptom_questions[key]['answer']}" for key in symptom_questions.keys() if symptom_questions[key]['answer'] is not None])
    bot.send_message(chat_id, summary + "\n" + response_summary)
    insert_into_database(questions, symptom_questions, user_responses[chat_id], username, user_id)

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    chat_id = message.chat.id
    user_id = message.from_user.id
    username = message.from_user.username
    print(f"Received message from {username} ({user_id}): {message.text}")

    if chat_id not in user_responses:
        user_responses[chat_id] = {}
        current_question_index[chat_id] = 0
        current_symptom_question_index[chat_id] = 0
        ask_next_question(chat_id)
        return

    if current_question_index[chat_id] == -1:
        response = message.text.lower()
        if response == "no":
            end_of_questions(chat_id, username, user_id)
        else:
            user_responses[chat_id]["symptoms"] = response
            current_question_index[chat_id] = len(questions)
            ask_symptom_related_questions(chat_id)
        return

    index = current_question_index[chat_id]
    question_keys = list(questions.keys())
    if index < len(question_keys):
        prev_question_key = question_keys[index]
        try:
            response = message.text
            response = validate_and_convert_input(response, questions[prev_question_key]["type"])
            if "chatgpt" in questions[prev_question_key]:
                corrected_response, is_valid = validate_with_chatgpt(questions[prev_question_key], response)
            elif "storegpt" in questions[prev_question_key]:
                corrected_response, is_valid = validate_with_storegpt(questions[prev_question_key], response)
            elif "yn" in questions[prev_question_key]:
                corrected_response, is_valid = validate_yes_or_no(questions[prev_question_key]["question"], response)
            elif "validation" in questions[prev_question_key]:
                corrected_response, is_valid = validate_and_correct_response(questions[prev_question_key]["question"], response, questions[prev_question_key].get("validation", []))
            else:
                corrected_response, is_valid = validate_and_correct(questions[prev_question_key]["question"], response)

            if is_valid:
                questions[prev_question_key]["answer"] = corrected_response
                user_responses[chat_id][prev_question_key] = corrected_response
                current_question_index[chat_id] += 1
                ask_next_question(chat_id)
            else:
                bot.send_message(chat_id, corrected_response)
        except ValueError as e:
            bot.send_message(chat_id, str(e) + "\nPlease try again.")
    else:
        symptom = user_responses[chat_id].get("symptoms", "").lower()
        # symptom = questions["symptoms"]["answer"]
        symptom_question_keys = list(symptom_questions.keys())
        symptom_index = current_symptom_question_index[chat_id]
        if symptom_index < len(symptom_question_keys):
            current_symptom_question_key = symptom_question_keys[symptom_index]
            if should_ask_symptom_question(symptom, current_symptom_question_key):
                try:
                    response = message.text
                    response = validate_and_convert_input(response, symptom_questions[current_symptom_question_key]["type"])
                    # if "chatgptduration" in symptom_questions[current_symptom_question_key]:
                    # corrected_response, is_valid = validate_symptom_duration_with_chatgpt(symptom_questions[current_symptom_question_key], response)
                    if "chatgpt" in symptom_questions[current_symptom_question_key]:
                        corrected_response, is_valid = validate_with_chatgpt(symptom_questions[current_symptom_question_key], response)
                    elif "storegpt" in symptom_questions[current_symptom_question_key]:
                        corrected_response, is_valid = validate_with_storegpt(symptom_questions[current_symptom_question_key], response)
                    # elif "durationgpt" in symptom_questions[current_symptom_question_key]:
                    #     corrected_response, is_valid = validate_with_durationgpt(symptom_questions[current_symptom_question_key]["question"], response)
                    elif "yn" in symptom_questions[current_symptom_question_key]:
                        corrected_response, is_valid = validate_yes_or_no(symptom_questions[current_symptom_question_key]["question"], response)
                    elif "validation" in symptom_questions[current_symptom_question_key]:
                        corrected_response, is_valid = validate_and_correct_response(symptom_questions[current_symptom_question_key]["question"], response, symptom_questions[current_symptom_question_key].get("validation", []))
                    else:
                        corrected_response, is_valid = validate_and_correct(symptom_questions[current_symptom_question_key]["question"], response)

                    if is_valid:
                        symptom_questions[current_symptom_question_key]["answer"] = corrected_response
                        current_symptom_question_index[chat_id] += 1
                        ask_symptom_related_questions(chat_id)
                    else:
                        bot.send_message(chat_id, corrected_response)
                except ValueError as e:
                    bot.send_message(chat_id, str(e) + "\nPlease try again.")
            else:
                current_symptom_question_index[chat_id] += 1
                ask_symptom_related_questions(chat_id)
        else:
            end_of_symptom_questions(chat_id)

# DATABASE PART

def insert_into_database(questions, symptom_questions, responses, username, user_id, db_name="responses.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("""SELECT count(name) FROM sqlite_master WHERE type='table' AND name='user_responses'""")
    if cursor.fetchone()[0] == 0:
        cursor.execute("""
            CREATE TABLE user_responses (
                username TEXT,
                user_id TEXT,
               buyformyself TEXT,
                drug_allergies TEXT,
                existing_conditions TEXT,
                current_medication TEXT,
                smoking TEXT,
                vaping TEXT,
                alcohol TEXT,
                occupation TEXT,
                recent_travel TEXT,
                unwell_contacts TEXT,
                pregnant TEXT,
                symptoms TEXT,
                medication TEXT,
                cold_duration TEXT,
                cold_fever TEXT,
                nasal_discharge TEXT,
                facial_pain TEXT,
                flu_duration TEXT,
                flu_fever TEXT,
                cough_duration TEXT,
                cough_phlegm TEXT,
                cough_blood TEXT,
                recent_cough TEXT,
                severe_cough TEXT,
                cough_relief TEXT,
                cough_wheezing TEXT,
                dust_exposure TEXT,
                rhinitis_duration TEXT,
                rhinitis_occurrence TEXT,
                rhinitis_exposure TEXT,
                sore_throat_duration TEXT,
                pollutant_exposure TEXT,
                runny_nose_duration TEXT,
                blocked_nose_duration TEXT,
                nausea_duration TEXT,
                nausea_timing TEXT,
                nausea_trigger TEXT,
                vomit_color TEXT,
                vomit_duration TEXT,
                diarrhea_occurrence TEXT,
                diarrhea_duration TEXT,
                diarrhea_color TEXT,
                eating_habit_diarrhea TEXT,
                abdominal_pain_diarrhea TEXT,
                indigestion_duration TEXT,
                sufficient_rest TEXT,
                active TEXT,
                eating_habit_indigestion TEXT,
                constipation_duration TEXT,
                eating_habit_constipation TEXT,
                abdominal_pain_constipation TEXT,
                cramp_duration TEXT,
                inflamed_duration TEXT,
                inflamed_part TEXT,
                inflamed_spread TEXT,
                fever TEXT,
                heartburn_duration TEXT,
                headache_duration TEXT,
                headache_part TEXT,
                eye_infection_duration TEXT,
                eye_redness_infection TEXT,
                eye_discharge TEXT,
                blink_pain TEXT,
                tired_eye_duration TEXT,
                eye_redness_tired TEXT,
                vision_blurry TEXT,
                insomnia_occurrence TEXT,
                fungal_duration TEXT,
                fungal_area TEXT,
                ulcer_duration TEXT,
                ulcer_description TEXT,
                ulcer_appearance TEXT,
                acne_occurrence TEXT,
                facial_product TEXT,
                acne_size TEXT,
                acne_inflamed TEXT,
                acne_pus TEXT
            )
            """)

    cursor.execute("""
        INSERT INTO user_responses (
            username,
            user_id,
            buyformyself,
            drug_allergies,
            existing_conditions,
            current_medication,
            smoking,
            vaping,
            alcohol,
            occupation,
            recent_travel,
            unwell_contacts,
            pregnant,
            symptoms,
            medication,
            cold_duration,
            cold_fever,
            nasal_discharge,
            facial_pain,
            flu_duration,
            flu_fever,
            cough_duration,
            cough_phlegm,
            cough_blood,
            recent_cough,
            severe_cough,
            cough_relief,
            cough_wheezing,
            dust_exposure,
            rhinitis_duration,
            rhinitis_occurrence,
            rhinitis_exposure,
            sore_throat_duration,
            pollutant_exposure,
            runny_nose_duration,
            blocked_nose_duration,
            nausea_duration,
            nausea_timing,
            nausea_trigger,
            vomit_color,
            vomit_duration,
            diarrhea_occurrence,
            diarrhea_duration,
            diarrhea_color,
            eating_habit_diarrhea,
            abdominal_pain_diarrhea,
            indigestion_duration,
            sufficient_rest,
            active,
            eating_habit_indigestion,
            constipation_duration,
            eating_habit_constipation,
            abdominal_pain_constipation,
            cramp_duration,
            inflamed_duration,
            inflamed_part,
            inflamed_spread,
            fever,
            heartburn_duration,
            headache_duration,
            headache_part,
            eye_infection_duration,
            eye_redness_infection,
            eye_discharge,
            blink_pain,
            tired_eye_duration,
            eye_redness_tired,
            vision_blurry,
            insomnia_occurrence,
            fungal_duration,
            fungal_area,
            ulcer_duration,
            ulcer_description,
            ulcer_appearance,
            acne_occurrence,
            facial_product,
            acne_size,
            acne_inflamed,
            acne_pus
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        username,
        user_id,
        responses.get("buyformyself"),
        responses.get("drug_allergies"),
        responses.get("existing_conditions"),
        responses.get("current_medication"),
        responses.get("smoking"),
        responses.get("vaping"),
        responses.get("alcohol"),
        responses.get("occupation"),
        responses.get("recent_travel"),
        responses.get("unwell_contacts"),
        responses.get("pregnant"),
        responses.get("symptoms"),
        responses.get("medication"),
        symptom_questions.get("cold_duration", {}).get("answer"),
        symptom_questions.get("cold_fever", {}).get("answer"),
        symptom_questions.get("nasal_discharge", {}).get("answer"),
        symptom_questions.get("facial_pain", {}).get("answer"),
        symptom_questions.get("flu_duration", {}).get("answer"),
        symptom_questions.get("flu_fever", {}).get("answer"),
        symptom_questions.get("cough_duration", {}).get("answer"),
        symptom_questions.get("cough_phlegm", {}).get("answer"),
        symptom_questions.get("cough_blood", {}).get("answer"),
        symptom_questions.get("recent_cough", {}).get("answer"),
        symptom_questions.get("severe_cough", {}).get("answer"),
        symptom_questions.get("cough_relief", {}).get("answer"),
        symptom_questions.get("cough_wheezing", {}).get("answer"),
        symptom_questions.get("dust_exposure", {}).get("answer"),
        symptom_questions.get("rhinitis_duration", {}).get("answer"),
        symptom_questions.get("rhinitis_occurrence", {}).get("answer"),
        symptom_questions.get("rhinitis_exposure", {}).get("answer"),
        symptom_questions.get("sore_throat_duration", {}).get("answer"),
        symptom_questions.get("pollutant_exposure", {}).get("answer"),
        symptom_questions.get("runny_nose_duration", {}).get("answer"),
        symptom_questions.get("blocked_nose_duration", {}).get("answer"),
        symptom_questions.get("nausea_duration", {}).get("answer"),
        symptom_questions.get("nausea_timing", {}).get("answer"),
        symptom_questions.get("nausea_trigger", {}).get("answer"),
        symptom_questions.get("vomit_color", {}).get("answer"),
        symptom_questions.get("vomit_duration", {}).get("answer"),
        symptom_questions.get("diarrhea_occurrence", {}).get("answer"),
        symptom_questions.get("diarrhea_duration", {}).get("answer"),
        symptom_questions.get("diarrhea_color", {}).get("answer"),
        symptom_questions.get("eating_habit_diarrhea", {}).get("answer"),
        symptom_questions.get("abdominal_pain_diarrhea", {}).get("answer"),
        symptom_questions.get("indigestion_duration", {}).get("answer"),
        symptom_questions.get("sufficient_rest", {}).get("answer"),
        symptom_questions.get("active", {}).get("answer"),
        symptom_questions.get("eating_habit_indigestion", {}).get("answer"),
        symptom_questions.get("constipation_duration", {}).get("answer"),
        symptom_questions.get("eating_habit_constipation", {}).get("answer"),
        symptom_questions.get("abdominal_pain_constipation", {}).get("answer"),
        symptom_questions.get("cramp_duration", {}).get("answer"),
        symptom_questions.get("inflamed_duration", {}).get("answer"),
        symptom_questions.get("inflamed_part", {}).get("answer"),
        symptom_questions.get("inflamed_spread", {}).get("answer"),
        symptom_questions.get("fever", {}).get("answer"),
        symptom_questions.get("heartburn_duration", {}).get("answer"),
        symptom_questions.get("headache_duration", {}).get("answer"),
        symptom_questions.get("headache_part", {}).get("answer"),
        symptom_questions.get("eye_infection_duration", {}).get("answer"),
        symptom_questions.get("eye_redness_infection", {}).get("answer"),
        symptom_questions.get("eye_discharge", {}).get("answer"),
        symptom_questions.get("blink_pain", {}).get("answer"),
        symptom_questions.get("tired_eye_duration", {}).get("answer"),
        symptom_questions.get("eye_redness_tired", {}).get("answer"),
        symptom_questions.get("vision_blurry", {}).get("answer"),
        symptom_questions.get("insomnia_occurrence", {}).get("answer"),
        symptom_questions.get("fungal_duration", {}).get("answer"),
        symptom_questions.get("fungal_area", {}).get("answer"),
        symptom_questions.get("ulcer_duration", {}).get("answer"),
        symptom_questions.get("ulcer_description", {}).get("answer"),
        symptom_questions.get("ulcer_appearance", {}).get("answer"),
        symptom_questions.get("acne_occurrence", {}).get("answer"),
        symptom_questions.get("facial_product", {}).get("answer"),
        symptom_questions.get("acne_size", {}).get("answer"),
        symptom_questions.get("acne_inflamed", {}).get("answer"),
        symptom_questions.get("acne_pus", {}).get("answer")
    ))

    conn.commit()
    conn.close()

bot.polling()


Received message from Jjjuuussshhh (5325434954): Hi
Received message from Jjjuuussshhh (5325434954): Yes
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): Student
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): No
Received message from Jjjuuussshhh (5325434954): Diarrhea
Received message from Jjjuuussshhh (5325434954): None
Received message from Jjjuuussshhh (5325434954): I'm not sure, sometimes it happens in the morning, sometimes at night, so it's quite random and hard for 

In [14]:
import sqlite3

def recreate_table(db_name="responses.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS user_responses")

In [9]:
!pip install pandas

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.
